In [81]:
# path = "./dataset/valid"
path = "./dataset/train"

In [82]:
import json
 
f = open(path+"/_annotations.coco.json")
data = json.load(f)
 
id_to_image_name = {}
for image in data["images"]:
    id_to_image_name[image["id"]] = image["file_name"]

image_name_to_annotations = {}
for annotation in data["annotations"]:
    image_name_to_annotations[id_to_image_name[annotation["image_id"]]] = annotation
f.close()

In [83]:
def parse_to_images(image):
    w, h = 512, 512
    data = np.zeros((h, w, 3), dtype=np.uint8)
    data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
    return Image.fromarray(image, 'RGB')

In [84]:
def show_top(images, start=0, end=10, annotations = None):
    c = 0
    for name, image in list(images.items())[start:end]:
        fig, ax = plt.subplots()
        ax.imshow(image)
        plt.axis(False)
        if annotations is not None and name in annotations.keys():
            print(name[:6])
            bbox = annotations[name]["bbox"]
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
        plt.show()

In [85]:
from PIL import Image
import glob
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches
 
# images_arr = []
# images_names = []
images_arr = {}
for f in glob.iglob(path+"/*.jpg"):
    images_arr[f[16:]] = np.asarray(Image.open(f))
    # images_arr.append(np.asarray(Image.open(f)))
    # images_names.append(f[16:])
# images_arr = np.array(images_arr)

images = {}
for name, image_arr in images_arr.items():
    images[name] = parse_to_images(image_arr)

start = 0
end = 0
show_top(images, start, end, image_name_to_annotations)